In [1]:
import numpy as np
import pandas as pd
from keras.layers import *
from keras.models import *
from keras.optimizers import *
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder

Using TensorFlow backend.


# Load data

The first cell loads the encoded images and splits them into two (home/away). The images are of shape 9x11x3 and are inputs for the siamese net. The second cell loads the match data. 

In [2]:
formations = np.load("data/formations.npy")

# Construct Siamese Model

In [3]:
def get_siamese_model(formation_input_shape, reduced_attr_input_shape):
    """
        Model architecture
    """
    
    # Define the tensors for the two input images
    home_formation_input = Input(formation_input_shape)
    away_formation_input = Input(formation_input_shape)
    
    attr_input = Input(reduced_attr_input_shape)
    
    
    # Convolutional Neural Network
    cnn = Sequential()
    cnn.add(Conv2D(16, (2,2), activation='relu', input_shape=formation_input_shape))
    cnn.add(MaxPooling2D())
    cnn.add(Flatten())
    cnn.add(Dense(64, activation='sigmoid'))
    
    # Generate the encodings (feature vectors) for the two images
    cnn_encoded_home = cnn(home_formation_input)
    cnn_encoded_away = cnn(away_formation_input)
    
    # attribute encoding MLP
    mlp0 = Sequential()
    mlp0.add(Dense(32, activation='sigmoid', input_shape=reduced_attr_input_shape))
    mlp0.add(Dense(32, activation='relu'))
    mlp0.add(Dense(64, activation='tanh'))
    
    mlp_encoded_attr = mlp0(attr_input)
    
    # prediction generating MLP
    concat = Concatenate()
    encoded_input = concat([mlp_encoded_attr, cnn_encoded_home, cnn_encoded_away])
    mlp1 = Sequential()
    mlp1.add(Dense(64, activation='tanh',input_shape=(64*3,)))
    mlp1.add(Dense(32, activation='relu'))
    mlp1.add(Dense(3,activation='softmax'))
    
    # Add a dense layer with a softmax unit to generate the probabilities of home and away team winning
    prediction = mlp1(encoded_input)
    
    # Connect the inputs with the outputs
    siamese_net = Model(inputs=[attr_input, home_formation_input, away_formation_input], outputs=prediction)
    optimizer = Adam(lr = 0.00005)
    siamese_net.compile(loss="categorical_crossentropy",optimizer=optimizer, metrics=['accuracy'])
    # return the model
    return siamese_net

# Train the Siamese Net

Train with k-fold cross validation. 

In [5]:
# split images into home/away teams
formations_home = formations[:, 0, :, :, :]
formation_away = formations[:, 1, :, :, :]
n = formations.shape[0]
m = int(n * .8)

formations_home_train = formations[:m, 0, :, :, :]
formations_away_train = formations[:m, 1, :, :, :]
formations_home_test = formations[m:, 0, :, :, :]
formations_away_test = formations[m:, 1, :, :, :]
y = pd.read_csv("data/y_resampled_formation.csv").values

for num_features in [5, 10, 25, 50, 100, 250, 500]:
    print("******************" + str(num_features) + "*********************")
    file_name_data_formation = "data/pca/" + str(num_features) + "/data_formation_" + str(num_features) + ".npy"
    file_name_test_data_formation = "data/pca/" + str(num_features) + "/test_data_formation_" + str(num_features) + ".npy"
    
    X = np.load(file_name_data_formation)
    reduced_attr_train = X[:m]
    reduced_attr_test = X[m:]
    label_train = y[:m]
    label_test = y[m:]
    siamese_net = get_siamese_model(formations_home.shape[1:], (X.shape[1],))
    siamese_net.fit(x=[reduced_attr_train, formations_home_train, formations_away_train],
                    y=label_train,
                    validation_data=([reduced_attr_test, formations_home_test, formations_away_test], label_test),
                    batch_size=8, epochs=10)
    # siamese_net.predict(x=[reduced_attr_train, formations_home_train, formations_away_train])
    
# attr_home_train = X[:m, :429]
# attr_away_train = X[:m, 429:]
# attr_home_test = X[m:, :429]
# attr_away_test= X[m:, 429:]


# # print(attr_home_train.shape)
# # print(attr_away_train.shape)
# label_train = y[:m]
# label_test = y[m:]


******************5*********************
Train on 18633 samples, validate on 4659 samples
Epoch 1/10
18633/18633 [==============================] - 4s 216us/step - loss: 1.0755 - acc: 0.4139 - val_loss: 1.3673 - val_acc: 0.0000e+00
Epoch 2/10
18633/18633 [==============================] - 4s 197us/step - loss: 1.0744 - acc: 0.4157 - val_loss: 1.3927 - val_acc: 0.0373
Epoch 3/10
18633/18633 [==============================] - 4s 205us/step - loss: 1.0740 - acc: 0.4162 - val_loss: 1.3529 - val_acc: 0.0000e+00
Epoch 4/10
18633/18633 [==============================] - 4s 202us/step - loss: 1.0734 - acc: 0.4154 - val_loss: 1.3652 - val_acc: 0.0000e+00
Epoch 5/10
18633/18633 [==============================] - 4s 197us/step - loss: 1.0733 - acc: 0.4147 - val_loss: 1.3403 - val_acc: 0.0000e+00
Epoch 6/10
18633/18633 [==============================] - 4s 215us/step - loss: 1.0729 - acc: 0.4160 - val_loss: 1.2989 - val_acc: 0.0000e+00
Epoch 7/10
18633/18633 [==============================] - 4s 2

Epoch 6/10
18633/18633 [==============================] - 4s 240us/step - loss: 1.0453 - acc: 0.4684 - val_loss: 1.4503 - val_acc: 0.2511
Epoch 7/10
18633/18633 [==============================] - 5s 273us/step - loss: 1.0412 - acc: 0.4753 - val_loss: 1.3924 - val_acc: 0.2483
Epoch 8/10
18633/18633 [==============================] - 4s 236us/step - loss: 1.0384 - acc: 0.4745 - val_loss: 1.4687 - val_acc: 0.2666
Epoch 9/10
18633/18633 [==============================] - 5s 260us/step - loss: 1.0357 - acc: 0.4752 - val_loss: 1.4618 - val_acc: 0.2501
Epoch 10/10
18633/18633 [==============================] - 5s 250us/step - loss: 1.0326 - acc: 0.4790 - val_loss: 1.5536 - val_acc: 0.2460
******************500*********************
Train on 18633 samples, validate on 4659 samples
Epoch 1/10
18633/18633 [==============================] - 6s 300us/step - loss: 1.0744 - acc: 0.4173 - val_loss: 1.4542 - val_acc: 0.0000e+00
Epoch 2/10
18633/18633 [==============================] - 5s 258us/step - l